## **0. What is `coupledbeam`?**

`coupledbeam` simulates the propagation of light through slowly-varying tapered waveguides using the coupled-mode approach. When applicable, this approach is typically much faster than alternate techniques like the finite difference beam propagation method (FD-BPM). `coupledbeam` is mostly written in Python, and has a completely Pythonic interface, with some parts written in Julia. In this notebook, we'll go over installation instructions, give some starting examples, and provide basic documentation.

## **1. Basics**

#### **1.1 How to install**
Right now, `coupledbeam` is not packaged (though it will be in the future). For now,  install by cloning the Github repo. You will need both a Python3 and a <a href="https://julialang.org/downloads/">Julia</a> installation. Below are the following dependencies:

**General**: `Gmsh` (download <a href="https://gmsh.info/">here</a>)

**Python**: `numpy`,`scipy`,`juliacall`,`wavesolve`,`pygmsh`,`meshio`,`matplotlib` <br> (All `pip` installable besides `wavesolve`, download <a href="https://github.com/jw-lin/wavesolve">here</a>)

**Julia**: `PythonCall`, `StaticArrays`, `GrundmannMoeller` <br>
To install the Julia packages, after cloning `coupledbeam`, start a Julia REPL in the root directory and run the following:

```
using Pkg
Pkg.activate("FEval")
Pkg.add("PythonCall")
Pkg.add("StaticArrays")
Pkg.add("GrundmannMoeller")
exit()
```

When updating, if a change has been made to the Julia source code, you will have to re-run the above (and most likely delete the auto-generated `Manifest.toml` file and`.CondaPkg` folder in the `FEval` directory beforehand, if they exist.)

#### **1.2 How it works**

Below we provide a high-level overview of how the code works.

**1.** To solve for the instantaneous eigenmodes of the waveguide at a given $z$ coordinate (which we define as the propagation direction), we use `wavesolve`, a Python package that applies the finite element method over quadratic triangle meshes.

**2.** To generate meshes corresponding to the waveguide refractive index profile at $z$, we use `pygmsh`/`Gmsh`. **Only waveguides with step-index profiles are currently supported**.

**3.** To simulate tapered waveguides, meshes are continuously evolved forward in $z$ by rescaling all mesh points. **Only tapered waveguides that can be represented this way are currently fully supported**. (Though it can be possible to simulate other kinds of waveguides with a little more work. I plan on extending functionality in the future. Probably will require an implementation of mesh generation with user-defined symmetries.)

**4.** In order to quickly evaluate a FE mode field at an arbitrary $(x,y)$ point, `coupledbeam` includes `FEval`, a simple Julia package that implements bounding volume hierarchy (BVH) trees. These structures are used to store mesh triangles, for faster lookup. (There are probably much better implementations out there which I might switch to in the future.)  

**5.** To quickly integrate functions involving FE modes (i.e. to take inner products), we use the `GrundmanMoeller` Julia package.

**6.** The initial value problem corresponding to the coupled-mode equations is solved using standard Runge-Kutta schemes, as implemented through `SciPy`.

#### **1.3 Code structure**

`coupledbeam` provides the following modules.

> `waveguide` : one of two main modules, used to define waveguides.

> `propagator` : the second main module, used to define propagation parameters and run propagations.

> `FEval` : the "finite element evaluation" module, comprised of Julia source code and a Python wrapper unless you want to resample eigenmodes (e.g. onto rectangular grids).

> `example.py` : an example Python scripts which simulates a 6-port photonic lantern. A similar example is covered in in Section 3.

## **2. How to use `coupledbeam`**

Broadly speaking, using `coupledbeam` is a two-step process: we first define a waveguide using the `waveguide` module, and then define a set of parameters for the numerical using the `propagator` module.


### **2.1 Defining waveguides**

#### **2.1.1** *the `Prim2D` class*

The most basic parent class that represents a refractive index geometry is a `Prim2D`. Each `Prim2D` specifies the boundary of a closed region - through an array of $(x,y)$ points - and a refractive index value corresponding to the interior that region. Physically, these boundaries represent material interfaces in the waveguide. A `Prim2D` is generically initialized through

`prim2D = waveguide.Prim2D(points,n)`

**Arguments**

1. `points` : an $N\times 2$ array of $(x,y)$ points corresponding to a material interface. (The first and last point are automatically connected.)
2.  `n` : the refractive index inside the region bounded by `points`. 

To simulate complex waveguide geometries, users are encouraged to define subclasses that inherit from `Prim2D`. Such classes can have there own `__init__()` functions, but should define a function `make_points()`, which takes in some set of args and returns an array `points`.  An example subclass of `waveguide.Prim2D` that is already implemented is `waveguide.Circle`.

#### **2.1.2** *the `Prim3D` class*

The next level in complexity is the `Prim3D` parent class, which is essentially a `Prim2D` combined with a function that dictates evolution with respect to $z$. A `Prim3D` is generically initialized as 

`prim3D = waveguide.Prim2D(prim2D,label)`

**Arguments**

1. `prim2D` : a `Prim2D` object representing the cross-section of the `Prim3D`
2. `label` : a user-specified identifying string to attach to the physical region bounded by `prim2D.points` (e.g. "core" or "cladding" for a step-index fiber.)

Inheriting classes must implement the function `update(z)`, which updates `prim2D` to the desired $z$ coordinate. An example subclass of `waveguide.Prim3D` is `waveguide.Pipe`, which can be used to represent geometries where the cross-section remains circular at all $z$.

#### **2.1.3** *the `Waveguide` class*

At the highest level, waveguides are defined through the `Waveguide` parent class. `Waveguide` objects are essentially nested lists of `Prim3D` objects: something like `[[p1,p2],[p3,p4,p5],[p6]]` where `p1,p2,...` are individual `Prim3D` object (the exact shape will vary). The refractive index geometry generated by the first set of primitives is overwritten by the next, and so on. A `Waveguide` is generically initialized as 

`wvg = Waveguide(prim3Dgroups)` 

where `prim3Dgroups` is the nested list of 3D primitives mentioned above.

All `Waveguide` objects have three main class functions:

> `Waveguide.update(z)` : this function updates the waveguide to a given $z$ coordinate, returning `None`. The base behavior of this function is to call `update` on every `Prim3D` in the waveguide.

> `Waveguide.make_mesh()` : this function is implemented by the base class and returns a quadratic triangle finite element mesh correspodning to the cross-sectional structure of the waveguide at the current $z$ coordinate.

> `Waveguide.assign_IOR()` : this *static* function returns a dictionary `IOR_dict` that maps each labelled region in the mesh to a refractive index value. The keys of this dictionary are the labels of the `Prim3D` objects composing the waveguide. Must be implemented by the inheriting class.


There is also a more advanced mesh-generating function, `Waveguide.make_mesh_bndry_ref()`, which is similar to the usual `make_mesh()` function with the added option of adaptively refining the mesh in regions near material interfaces.

#### **2.1.4** *an example waveguide - the `PhotonicLantern` class*

An example of a `Waveguide` subclass is the `PhotonicLantern` class. This class of waveguide is composed of three groups of primitives: a single outer `Pipe` corresponding to lantern jacket, a single inner `Pipe` corresponding to the lantern cladding, and a set of small `Pipe`s embedded within the cladding - the cores. This class has a custom `__init__()` function which automatically generates and arranges `Pipe`s based on parameters like core radius, cladding radius, and core position(s). See `waveguide.PhotonicLantern` for more details. 

#### **2.1.5** *utility functions for `Waveguides`*
Below we list some useful functions implemented by the base `Waveguide` class.

> `plot_boundaries()` : plot all boundaries (material interfaces) in the waveguide, at the currently updated $z$ coordinate.

> `plot_mesh()` : generate and plot a mesh for the waveguide. You can also explicitly pass in a mesh, produced e.g. by `Waveguide.make_mesh()`, as well as refractive index information, through the optional args `mesh` and `IOR_dict`.

### **2.2 Propagation - the `Propagator` class**

Propagation is controlled via the `Propagator` class. 

#### **2.2.1** *initialization*

Below is the pattern for initialization: 

`prop = propagator.Propagator(wl,wvg,Nmax,save_dir)`

**Arguments**

1.  `wl` : the propagation wavelength (in the same spatial unit that the waveguide geometry is defined in)
2. `wvg` : a `Waveguide` object
3. `Nmax` : the number of propagating modes in this waveguide (we cannot at the moment treat radiative modes)
4. `save_dir` : the local path for where data should be saved. The default is `"./data"`. Folders will be created if they are not found.

#### **2.2.2** *"characterizing" the waveguide*

Before we can apply coupled-mode theory to simulate the waveguide we have loaded, we need to pre-compute the following waveguide properties, as a function of $z$:
* the instanteous eigenmodes
* the instanteous eigenvalues
* the cross-coupling matrix (formed from the inner product of eigenmodes and eigenmode derivatives.)

This step takes the bulk of the time when it comes to waveguide simulation; however, once we have the above, propagation of any guided wavefront through the waveguide is fast. To do this setup, we use the function `prop.prop_setup()`, which will adaptively step forwards in $z$ and compute the required items. Below is the function pattern:

`prop.prop_setup(zi,zf)`

**Arguments**

1. `zi` : the starting $z$ coordinate, which does not have to be 0.
2. `zf` : the ending $z$ coordinate, which does not have to be the maximum extent of the waveguide.

There are also a number of optional arguments related to computation details.

**Optional arguments**

3. `tol` : this tolerance parameter controls the adaptive stepping. Smaller = more careful stepping. Values are typically in the 1e-4 to 1e-6 range (default 1e-5).
4. `dz0` : this is the fixed $z$ step used to estimate derivatives via centered finite difference. Default is 0.1, but might need to be changed depending on your choice of spatial units.
5. `min_zstep` : this is the minimum `z` step that can be chosen by the adaptive scheme. Computation will halt when the stepsize drops below this value. Default 1. Certain meshes sometimes perform poorly an trigger a halt - in these cases you can try remeshing with different parameters and rerunning. (Unfortunately I don't have a better solution atm!)
6. `save` : set to True to write all computation results to file. These can be loaded later with `prop.load(tag)`, where `tag` is defined as below
7. `tag` : a string tag to attach to output files so that they can be loaded later.
8. `fixed_degen` : a nested list of indices (starting from 0) which identify degenerate mode groups. For instance, if we know *a priori* that modes (1,2) and modes (3,4) form degenerate pairs, we could pass in `fixed_degen=[[1,2],[3,4]]`. This can speed up computation (and resolve issues with `min_zstep`).
9. `fixed_step` : set this to a numeric value to used a fixed $z$ step, bypassing the adaptive scheme.
10. `mesh` : explicitly pass in a mesh (generated from `Waveguide.make_mesh()` or similar) if you don't want to use an auto-generated one.


#### **2.2.3** *propagation*

Once `prop_setup()` has been run, or files produced by a previous setup have been loaded with `prop.load(tag)`, we can simulate light propagation. The first step is to quickly generate interpolation functions for the eigenmodes, eigenvalues, and cross-coupling matrix over $z$. This is done with:

`prop.make_interp_funcs()`

From here, we can define a launch field, expressed in the basis of the initial eigenmodes of the waveguide, e.g. 

`u0 = [1,0,0,0,0,0]`

and the propagate the field using `prop.propagate()`. The pattern is as follows:

`z,u,uf = prop.propagate(u0,zf,WKB)`

Arguments 

1. `u0` : the launch wavefront, expressed in the basis of the initial eigenmodes.
2. `zf` : the ending $z$ coordinate for propagation
3. `WKB` : (optional), a boolean which controls whether or not an additional correction is included in the coupled-mode equations (default `True`)

Returns

1. `z` : an array of $z$ values selected by the IVP solver (`scipy.integrate.solve_ivp`, RK23 scheme).
2. `u` : an `Nmax` $\times$ `M` array of mode amplitudes computed at each of the `M` values of `z` (with fast $e^{i\beta z}$ oscillation factored *out*).  
3. `uf` : the final mode amplitudes (with fast $e^{i\beta z}$ oscillation factored *in* - these are the actual complex-valued mode amplitudes at `zf`).


## **3. Worked example: 6-port photonic lantern**
Below I include some code which uses `coupledbeam` to simulate the propagation of the ${\rm LP}_{01}$ mode through a "standard" (uniform core size) 6-port photonic lantern. Below, we define all the waveguide and simulation parameters.

In [7]:
### we first define some lantern params ###

import numpy as np

wl = 1.55                       # wavelength, um
taper_factor = 8.               # relative scale factor between frontside and backside waveguide geometry    
rcore = 2.2/taper_factor        # radius of tapered-down single-mode cores at frontside, um
rclad = 10                      # radius of cladding-jacket interface at frontside, um
rjack = 30                      # radius of outer jacket boundary at frontside, um
z_ex = 40000                    # lantern length, um

nclad = 1.444                   # cladding refractive index
ncore = nclad + 8.8e-3          # SM core refractive index
njack = nclad - 5.5e-3          # jacket refractive index

### define the initial (x,y) locations of the lantern cores

t = 2*np.pi/5                   
initial_offset = rclad*2/3 # this is the radial distance of the outer lantern cores from the center
xpos_i = [0] + [initial_offset*np.cos(i*t) for i in range(5)]
ypos_i = [0] + [initial_offset*np.sin(i*t) for i in range(5)]
core_pos = np.array([xpos_i,ypos_i]).T 

### mesh resolution params

core_res = 80                       # no. of line segments to use to resolve the core-cladding interface(s)
clad_res = 150                      # no. of line segments to use to resolve the cladding-jacket interface
jack_res = 30                       # no. of line segments to form the outer jacket boundary
clad_mesh_size = 1.0                # mesh size (triangle side length) to use in the cladding region
core_mesh_size = 0.1                # mesh size (triangle side length) to use inside the cores

### propagator params

tol = 1e-5              # adaptive z-stepping tolerance
dz0 = 0.1               # fixed z-step to use for finite-differences
fixed_degen = []        # degenerate mode groups - we leave this empty for now.

std_rcores = np.array([rcore]*6) # arry of core radii. note that we have set all radii equal -> standard lantern.
ncores = [ncore]*6 # array of core refractive indices

Next, we initialize our `PhotonicLantern` object (yes there are a lot of args, I need to work on this ...)

In [8]:
import waveguide
lant = waveguide.PhotonicLantern(core_pos,std_rcores,rclad,rjack,ncores,nclad,njack,z_ex,taper_factor,core_res,clad_res,jack_res,core_mesh_size,clad_mesh_size)

Let's use the built-in `plot_boundary` and `update` functions to look at the waveguide structure. 

In [ ]:
lant.update(0) # set z = 0, lantern entrance
lant.plot_boundaries()

Next, we initialize a `Propagator` object ...

In [9]:
import propagator

prop = propagator.Propagator(wl,lant,6)

Compute the eigenmodes, eigenvalues, and cross-coupling matrices of the waveguide for $z\in $[0,`z_ex`], and save it to `"test"`.

In [10]:
# this'll take like 45 minutes ...

output = prop.prop_setup(0,z_ex,save=True,tag="test",tol=tol,fixed_degen=fixed_degen,dz0=dz0)

# can also load with 
# prop.load(tag="test")

generating mesh...
number of mesh points:  7443
starting computation ...
current z: 40000 / 40000 ; current zstep: 640.0                time elapsed:  2933.5085451602936


([0,
  10,
  20,
  30,
  40,
  60,
  100,
  180,
  260,
  340,
  420,
  500,
  580,
  740,
  900,
  1060,
  1220,
  1540,
  1860,
  2180,
  2340.0,
  2500.0,
  2580.0,
  2660.0,
  2740.0,
  2900.0,
  3220.0,
  3860.0,
  5140.0,
  5460.0,
  5780.0,
  6100.0,
  6420.0,
  7060.0,
  8340.0,
  8980.0,
  9300.0,
  9620.0,
  9940.0,
  10260.0,
  10580.0,
  10900.0,
  11060.0,
  11220.0,
  11380.0,
  11540.0,
  11700.0,
  11860.0,
  12020.0,
  12180.0,
  12340.0,
  12500.0,
  12660.0,
  12820.0,
  12980.0,
  13140.0,
  13460.0,
  13620.0,
  13780.0,
  13940.0,
  14100.0,
  14260.0,
  14420.0,
  14580.0,
  14740.0,
  14900.0,
  15220.0,
  15380.0,
  15540.0,
  15700.0,
  15860.0,
  16180.0,
  16820.0,
  16980.0,
  17060.0,
  17140.0,
  17220.0,
  17380.0,
  17540.0,
  17860.0,
  18500.0,
  19140.0,
  19780.0,
  20420.0,
  21060.0,
  22340.0,
  23620.0,
  24260.0,
  24580.0,
  24900.0,
  25060.0,
  25220.0,
  25380.0,
  25700.0,
  26340.0,
  26660.0,
  26980.0,
  27300.0,
  27620.0,
  27940.0,
 

Let's look at the effective indices of the 6 waveguide modes as function of $z$, as well as the cross-coupling matrix, which tells us which modes can potentially cross-couple during propagation, and where the cross-coupling will occur.

Make the interpolation functions

In [11]:
prop.make_interp_funcs()

TypeError: list indices must be integers or slices, not tuple

Finally, propagate the ${\rm LP}_{01}$ mode (which has the highest eigenvalue and is therefore the first eigenmode at the waveguide entrance).

In [ ]:
u0 = [1,0,0,0,0,0]

z,u,uf = prop.propagate(u0,z_ex,WKB=True)

print("final mode amplitudes: ")
print(np.abs(uf))
print("final mode powers: ")
print(np.power(np.abs(uf),2))

import matplotlib.pyplot as plt
for i in range(6): # plotting evolution in mode power
    plt.plot(z,np.power(np.abs(u[i]),2),label='mode '+str(i))

plt.plot(z,np.sum(np.power(np.abs(u),2),axis=0),color='k',zorder=-100,label="total power",ls='dashed')
plt.title('LP01 propagation - std PL')

plt.xlabel(r'$z$ (um)')
plt.ylabel("power")

plt.legend(loc='best')
plt.show()